In [428]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jun  5 13:37:54 2019

@author: aw1001
"""
import time
from datetime import datetime
import cv2
import base64
import pandas as pd
import json
import requests
import datetime
import numpy as np
import datetime
import re
import dask.dataframe as dd
import sqlite3 as sqlite


url_cebrace_salvar = "http://localhost:22700/api/ocorrencia/salvar"
url_cebrace_update = "http://localhost:22700/api/ocorrencia/id/adicionar-foto"
#headers_cebrace = {"content-type": "application/x-www-form-urlencoded"}

In [429]:
'import bd_connect

SyntaxError: EOL while scanning string literal (<ipython-input-429-78b9ac827b88>, line 1)

In [395]:
conn = sqlite.connect('safety_camera.db')
cursor = conn.cursor()
df = pd.read_sql_query("SELECT * FROM images Where status = 10", conn, index_col =None)

In [396]:
#conexão com o webservice
url_cebrace_salvar = "http://sistemas.cebrace.com.br/EzTask.WebAPI/api/ocorrencia/salvar"
url_cebrace_update = "http://sistemas.cebrace.com.br/EzTask.WebAPI/api/ocorrencia/update_foto"

def connect_webservice_cebrace(username, password):
    url_cebrace_connect = "http://sistemas.cebrace.com.br/EzTask.WebAPI/api/token"
    headers_cebrace_connect = { "content-type": "application/x-www-form-urlencoded" }
    data_cebrace_connect = { "grant_type": "password", "username": username, "password": password }
    response_conect = requests.post(url_cebrace_connect, data = data_cebrace_connect, headers = headers_cebrace_connect)
    response_conect_json = response_conect.json()

    return response_conect_json["access_token"], response_conect_json["expires_in"]

access_token, expires_in = connect_webservice_cebrace("KEYRUSAPI", "SW2019")
headers_cebrace = { "content-type": "application/json", "Authorization": ("Bearer " + access_token) }

In [397]:
#dataframe =  'bancodeImagens1.csv'
#df =  pd.read_csv(dataframe,
                  #sep = ',', 
                  #error_bad_lines = False,
                #encoding='utf-8')

In [398]:
#df = df.reset_index(drop=True)
df.head()

,idimage,idcamera,acquisition_time,update_time,status,json_returned,eztask_return,raw_image,processed_image,anomaly_group,accuracy_check
0,4,4,2019-06-25 08:02:53,2019-06-28 10:53:52,10,"{""item_0"": {""top"": 387, ""left"": 1184, ""class"":...","{""id"": 2054, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0
1,11,4,2019-06-25 08:02:56,2019-06-28 10:53:52,10,"{""item_0"": {""top"": 387, ""left"": 1184, ""class"":...","{""id"": 2055, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0
2,18,4,2019-06-25 08:02:59,2019-06-28 10:53:53,10,"{""item_0"": {""top"": 389, ""left"": 1187, ""class"":...","{""id"": 2056, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0
3,25,4,2019-06-25 08:03:02,2019-06-28 10:53:53,10,"{""item_0"": {""top"": 389, ""left"": 1190, ""class"":...","{""id"": 2057, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0
4,32,4,2019-06-25 08:03:05,2019-06-28 10:53:54,10,"{""item_0"": {""top"": 389, ""left"": 1190, ""class"":...","{""id"": 2058, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0


In [413]:
now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print('acquisition_time XXXXXXX', df['acquisition_time'].iloc[:1][0])
df['acquisition_time'] = pd.to_datetime(df.acquisition_time)
df['acquisition_time'] = df['acquisition_time'].dt.floor('120s')
df['acquisition_time_group'] = df['acquisition_time'].factorize()[0]
agrupamentosCamera = df.groupby('idcamera')
#resultado = df.groupby(['idcamera'])
#resultado = resultado.apply(lambda busca :  busca[busca['acquisition_time_trunc'] == busca['acquisition_time_trunc'].agg(lambda x:x.value_counts().index[0])])
#df[df['acquisition_time_group'] == 0]
print(len(set(df['acquisition_time_group'])))

acquisition_time XXXXXXX 2019-06-25 08:02:00
9


In [422]:
df.iloc[:76]

,idimage,idcamera,acquisition_time,update_time,status,json_returned,eztask_return,raw_image,processed_image,anomaly_group,accuracy_check,acquisition_time_group
0,4,4,2019-06-25 08:02:00,2019-06-28 10:53:52,10,"{""item_0"": {""top"": 387, ""left"": 1184, ""class"":...","{""id"": 2054, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0,0
1,11,4,2019-06-25 08:02:00,2019-06-28 10:53:52,10,"{""item_0"": {""top"": 387, ""left"": 1184, ""class"":...","{""id"": 2055, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0,0
2,18,4,2019-06-25 08:02:00,2019-06-28 10:53:53,10,"{""item_0"": {""top"": 389, ""left"": 1187, ""class"":...","{""id"": 2056, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0,0
3,25,4,2019-06-25 08:02:00,2019-06-28 10:53:53,10,"{""item_0"": {""top"": 389, ""left"": 1190, ""class"":...","{""id"": 2057, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0,0
4,32,4,2019-06-25 08:02:00,2019-06-28 10:53:54,10,"{""item_0"": {""top"": 389, ""left"": 1190, ""class"":...","{""id"": 2058, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
71,1652,7,2019-06-26 18:38:00,2019-06-28 08:16:58,10,"{""item_0"": {""top"": 335, ""left"": 1477, ""class"":...","{""id"": 1892, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0,6
72,1659,7,2019-06-26 18:38:00,2019-06-28 08:16:58,10,"{""item_0"": {""top"": 335, ""left"": 1477, ""class"":...","{""id"": 1893, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0,6
73,1666,7,2019-06-26 18:38:00,2019-06-28 08:16:59,10,"{""item_0"": {""top"": 335, ""left"": 1477, ""class"":...","{""id"": 1894, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0,6
74,1673,7,2019-06-26 18:38:00,2019-06-28 08:16:59,10,"{""item_0"": {""top"": 335, ""left"": 1477, ""class"":...","{""id"": 1895, ""mensagem"": ""Ocorrência cadastrad...",/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,None,0.0,6


In [425]:
'''
Factorize, retorna o valor distinto dentro do conjunto de dados.
'''
# itera em cada grupo
for indiceAgrupamento, agrupamentoCamera in agrupamentosCamera:  
    #print(agrupamentoCamera.iterrows())
    
    primeiroRegistroCamera = agrupamentoCamera
    #print(agrupamentoCamera.iterrows())

    idcamera = (primeiroRegistroCamera['idcamera'])
    #print(type(idcamera))
    #acquisition_time = str(primeiroRegistroCamera['acquisition_time'])
    
    acquisition_time_formatted = pd.to_datetime(str(primeiroRegistroCamera['acquisition_time'].values[0])).strftime('%Y-%m-%d %H:%M:%S')
    
    update_time = primeiroRegistroCamera['update_time']
    status = primeiroRegistroCamera['status']
    json_returned = primeiroRegistroCamera['json_returned']
    anomalies = json_returned
    anomaly_txt= ""
    
    for item in anomalies:
        if "safety" in item:
            anomaly_txt += "\nAndando fora da faixa de segurança"
        elif re.findall(r'\b(\w*no helmet\w*)\b', item ) == ['no helmet']:
            anomaly_txt += "\nFalta de capacete"
        elif re.findall(r'\b(\w*no vest\w*)\b', item ) == ['no vest']:
            anomaly_txt += "\nFalta de colete"
    
    eztask_return = primeiroRegistroCamera['eztask_return']
    raw_image = primeiroRegistroCamera['raw_image']
    processed_image = primeiroRegistroCamera['processed_image']
    anomaly_group = primeiroRegistroCamera['anomaly_group']
    accuracy_check = primeiroRegistroCamera['accuracy_check']
    
    jpg_as_text = str(primeiroRegistroCamera['processed_image'])
    jpg_as_text = jpg_as_text.encode("UTF-8")
    jpg_as_text = base64.b64encode(jpg_as_text).decode('utf-8')
    
    data_cebrace =  {
        "dataInicial": acquisition_time_formatted,
        "dataFinal":now, 
        "camera": ("Unidade JCR - Câmera: " + str(idcamera) ),
        "anomalia": anomaly_txt,
        "anexo": jpg_as_text
    }
    
    
    data_cebrace = json.dumps(data_cebrace)
    response_cebrace_salvar = requests.post(url_cebrace_salvar, data = data_cebrace, headers = headers_cebrace)
    print(response_cebrace_salvar.text)
    
    if response_cebrace_salvar.status_code == 200:
        ocorrenciaSalvaEzTask = json.loads(response_cebrace_salvar.text)
        #print(ocorrenciaSalvaEzTask['id'])
        
        imagensRestantesCamera = agrupamentoCamera
        print(imagensRestantesCamera)
        
        for indice , imagemRestante in imagensRestantesCamera.iterrows(): 

            #dadosEnvioImagemRestate =  json.dumps({"atividadeID": ocorrenciaSalvaEzTask['id'],"anexo" : jpg_as_text})
            idocorrencia =  str(ocorrenciaSalvaEzTask['id'])
            print(idocorrencia)
            url_cebrace_update = url_cebrace_update.replace('id',idocorrencia)
            print(url_cebrace_update)
            dadosEnvioImagemRestate =  json.dumps({"anexo" : jpg_as_text})
            response_cebrace_update = requests.post(url_cebrace_update,data = dadosEnvioImagemRestate,headers = headers_cebrace)
            
            respostaDadosEnvioImagemRestate =  json.loads(response_cebrace_update.text)
            #print("Resposta do sistema",respostaDadosEnvioImagemRestate['mensagem'])
    else:
        break
            

    


{
  "id": 5231,
  "mensagem": "Ocorrência cadastrada com sucesso."
}
    idimage  idcamera    acquisition_time          update_time  status  \
76     1695         1 2019-06-27 08:44:00  2019-06-28 08:17:00      10   

                                        json_returned  \
76  {"item_0": {"top": 911, "left": 389, "class": ...   

                                        eztask_return  \
76  {"id": 1897, "mensagem": "Ocorrência cadastrad...   

                                            raw_image  \
76  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...   

                                      processed_image anomaly_group  \
76  bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...          None   

    accuracy_check  acquisition_time_group  
76             0.0                       7  
5231
http://sistemas.cebrace.com.br/EzTask.WebAPI/api/ocorrencia/update_foto
{
  "message": "System.Data.Entity.Infrastructure.DbUpdateException: An error occurred while updating the entries. See the inne

In [424]:
print(f"{i} imagens processadas. Aplicação Finalizada!")

0 imagens processadas. Aplicação Finalizada!
